In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Function to calculate Newey-West variance
def newey_west_variance(differences, lag=1):
    """
    Computes the Newey-West variance for the series of differences.
    
    Parameters:
    - differences: Array-like of differences (e.g., error differences between two models)
    - lag: Maximum lag to use for the Newey-West estimator (default is 1)
    
    Returns:
    - Newey-West variance of the differences
    """
    X = np.ones(len(differences))  # Constant term
    ols_model = sm.OLS(differences, X).fit(cov_type='HAC', cov_kwds={'maxlags': lag})
    return ols_model.bse[0]**2  # Variance (squared standard error of the constant term)

# Function to compute DM test statistic
def dm_test_statistic(errors1, errors2, lag=1):
    """
    Computes the Diebold-Mariano test statistic between two sets of errors.
    
    Parameters:
    - errors1: First set of errors (e.g., from the first model)
    - errors2: Second set of errors (e.g., from the second model)
    - lag: Maximum lag to use for the Newey-West estimator (default is 1)
    
    Returns:
    - DM test statistic
    """
    # Step 1: Square both sets of errors
    squared_errors1 = np.square(errors1)
    squared_errors2 = np.square(errors2)

    # Step 2: Calculate the difference between the squared errors
    squared_diff = squared_errors1 - squared_errors2

    # Step 3: Calculate the average difference (mean of d_t)
    mean_diff = np.mean(squared_diff)

    # Step 4: Calculate Newey-West variance
    nw_variance = newey_west_variance(squared_diff, lag)

    # Step 5: Compute the DM test statistic
    dm_statistic = mean_diff / np.sqrt(nw_variance)
    
    return dm_statistic

# Load the CSV files
adhbs_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/adhbs.csv')
ols_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/ols.csv')
enet_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/enet.csv')
glm_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/glm.csv')
rf_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/rf.csv')
xgb_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/xgb.csv')
nn_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/nn.csv')
ahbs_nn_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/ahbs_nn.csv')
ahbs_xgb_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/ahbs_xgb.csv')
ahbs_rf_dm_test_data = pd.read_csv('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/ahbs_rf.csv')


# Store the data in a dictionary for easier access
models = {
    "ADHBS": adhbs_dm_test_data['Daily Avg Errors'],
    "OLS": ols_dm_test_data['Daily Avg Errors'],
    "ENet": enet_dm_test_data['Daily Avg Errors'],
        "GLM": glm_dm_test_data['Daily Avg Errors'],
    "XGBoost": xgb_dm_test_data['Daily Avg Errors'],
     "RF": rf_dm_test_data['Daily Avg Errors'],
       "NN": nn_dm_test_data['Daily Avg Errors']
    # "AHBS + NN": ahbs_nn_dm_test_data['Daily Avg Errors'],
    #  "AHBS + XGBoost": ahbs_xgb_dm_test_data['Daily Avg Errors'],
    #  "AHBS + RF": ahbs_rf_dm_test_data['Daily Avg Errors']

}

# Prepare an empty DataFrame to store the results
dm_table = pd.DataFrame(index=models.keys(), columns=models.keys())

# Perform pairwise DM tests
for model1_name, errors1 in models.items():
    for model2_name, errors2 in models.items():
        if model1_name != model2_name:
            # Calculate DM test statistic
            dm_statistic = dm_test_statistic(errors1, errors2)
            dm_table.loc[model2_name, model1_name] = round(dm_statistic, 2)

# Display the resulting DM test table
print(dm_table)

# Optionally, save the DM test table to a CSV file
file_path = '/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/performence_evaluation/dm_test_results.csv'
dm_table.to_csv(file_path)
print(f"DM test results saved to {file_path}")


         ADHBS   OLS  ENet   GLM XGBoost    RF     NN
ADHBS      NaN  0.45  -1.7 -5.11   -8.55 -8.87 -10.07
OLS      -0.45   NaN -1.85 -6.18   -6.73 -7.01  -8.97
ENet       1.7  1.85   NaN -2.04   -6.35 -6.79  -5.64
GLM       5.11  6.18  2.04   NaN   -3.91 -4.25  -8.24
XGBoost   8.55  6.73  6.35  3.91     NaN -0.52  -0.81
RF        8.87  7.01  6.79  4.25    0.52   NaN  -0.71
NN       10.07  8.97  5.64  8.24    0.81  0.71    NaN
DM test results saved to /Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/performence_evaluation/dm_test_results.csv


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_53032/1197381616.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return ols_model.bse[0]**2  # Variance (squared standard error of the constant term)


# DB test comparison full vs reduced


In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Function to calculate Newey-West variance
def newey_west_variance(differences, lag=1):
    """
    Computes the Newey-West variance for the series of differences.
    """
    X = np.ones(len(differences))  # Constant term
    ols_model = sm.OLS(differences, X).fit(cov_type='HAC', cov_kwds={'maxlags': lag})
    return ols_model.bse[0]**2  # Variance (squared standard error of the constant term)

# Function to compute DM test statistic
def dm_test_statistic(errors1, errors2, lag=1):
    """
    Computes the Diebold-Mariano test statistic between two sets of errors.
    """
    squared_errors1 = np.square(errors1)
    squared_errors2 = np.square(errors2)
    squared_diff = squared_errors1 - squared_errors2
    mean_diff = np.mean(squared_diff)
    nw_variance = newey_west_variance(squared_diff, lag)
    dm_statistic = mean_diff / np.sqrt(nw_variance)
    return dm_statistic

# Load reduced model data
reduced_model_files = {
    "OLS": "ols.csv",
    "ENet": "enet.csv",
    "GLM": "glm.csv",
    "XGBoost": "xgb.csv",
    "RF": "rf.csv",
    "NN": "nn.csv"
}

# Load full model data (appending "_all" to filenames)
full_model_files = {k: v.replace('.csv', '_all.csv') for k, v in reduced_model_files.items()}

# Load errors for both reduced and full models
base_path = '/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/Notebook_modeling/db_performance_evaluation/'
reduced_models = {name: pd.read_csv(f'{base_path}{file}')['Daily Avg Errors'] for name, file in reduced_model_files.items()}
full_models = {name: pd.read_csv(f'{base_path}{file}')['Daily Avg Errors'] for name, file in full_model_files.items()}

# Compute DM statistics for each model
dm_results = {}
for model_name in reduced_models.keys():
    reduced_errors = reduced_models[model_name]
    full_errors = full_models[model_name]
    dm_stat = dm_test_statistic(reduced_errors, full_errors)
    dm_results[model_name] = dm_stat

# Convert results to DataFrame for display
dm_results_df = pd.DataFrame.from_dict(dm_results, orient='index', columns=["DM Statistic"])
dm_results_df


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_53032/2104540945.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return ols_model.bse[0]**2  # Variance (squared standard error of the constant term)


,DM Statistic
OLS,2.163842
ENet,2.931819
GLM,-15.500484
XGBoost,-1.403957
RF,-6.717921
NN,-3.365817
